<a href="https://colab.research.google.com/github/adryduty/computer-vision-cat-project/blob/main/Training_comp_vision_cat(11_June).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The following chunk allows you to create a yolov5 folder with all the things you need.

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12228, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 12228 (delta 15), reused 20 (delta 8), pack-reused 12195
Receiving objects: 100% (12228/12228), 11.98 MiB | 31.45 MiB/s, done.
Resolving deltas: 100% (8459/8459), done.
/content/yolov5
     |████████████████████████████████| 596 kB 30.6 MB/s 
     |████████████████████████████████| 145 kB 40.3 MB/s 
     |████████████████████████████████| 178 kB 37.9 MB/s 
     |████████████████████████████████| 1.1 MB 29.0 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 138 kB 54.7 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import numpy as np    
from PIL import Image
import pickle
from google.colab import files, drive
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import time
import matplotlib.patches as patches
import tensorflow as tf
import pandas as pd
import os
import shutil
from math import floor
from sklearn.model_selection import train_test_split

# Recreating all the directories of roboflow before applying the yolov5

### The following chunk creates empty folders

In [ ]:
path = "/content/yolov5/Gatto-1"
os.mkdir(path)

path = "/content/yolov5/Gatto-1/test"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/validation"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/training"
os.mkdir(path)

path = "/content/yolov5/Gatto-1/test/images"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/validation/images"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/training/images"
os.mkdir(path)

path = "/content/yolov5/Gatto-1/test/labels"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/validation/labels"
os.mkdir(path)
path = "/content/yolov5/Gatto-1/training/labels"
os.mkdir(path)

In [ ]:
def file_path_changer(source, dest): 
  '''
  This function moves the files from the path 'source' to the path 'dest'
  '''
  shutil.move(source, dest)


### Store in df the dataset with the labels and other data of all the pictures we have (notice that you have first to import it).

In [ ]:
df = pd.read_csv("/content/label_242.csv")

### Split in training, validation and testing set

In [ ]:
train, test = train_test_split(df, train_size = 0.7, test_size = 0.3, random_state = 42)

validation, test = train_test_split(test, train_size = 0.66, test_size = 0.34, random_state = 42)

train_labels = train.iloc[:, 1:].values
valid_labels = validation.iloc[:, 1:].values
test_labels = test.iloc[:, 1:].values

train_im_list = list(train.iloc[:,0])
valid_im_list = list(validation.iloc[:, 0])
test_im_list = list(test.iloc[:, 0])

### The following chunk saves labels and image names in the proper folders (after having uploaded by hand)

In [ ]:
for index in range(len(train_im_list)):
  file_name = "/content/yolov5/Gatto-1/training/labels/" + list(train_im_list)[index].split("/")[2].split(".")[0] + ".txt"
  f = open(file_name, "w")
  f.write(str(train_labels[index])[1:-1])
  f.close()


for index in range(len(valid_im_list)):
  file_name = "/content/yolov5/Gatto-1/validation/labels/" + list(train_im_list)[index].split("/")[2].split(".")[0] + ".txt"
  f = open(file_name, "w")
  f.write(str(train_labels[index])[1:-1])
  f.close()


for index in range(len(test_im_list)):
  file_name = "/content/yolov5/Gatto-1/test/labels/" + list(train_im_list)[index].split("/")[2].split(".")[0] + ".txt"
  f = open(file_name, "w")
  f.write(str(train_labels[index])[1:-1])
  f.close()

### Move the images from /content/imageName.jpg to the proper directory

In [ ]:
for source in train_im_list:
  dest = "/content/yolov5/Gatto-1/training/images"
  file_path_changer(source, dest)

for source in valid_im_list:
  dest = "/content/yolov5/Gatto-1/validation/images"
  file_path_changer(source, dest)

for source in test_im_list:
  dest = "/content/yolov5/Gatto-1/test/images"
  file_path_changer(source, dest)

### The following chunk creates the yaml

In [ ]:
data=open("/content/yolov5/Gatto-1/data.yaml", "w")
data.write("names:\n- Ghiri\nnc: 1\ntrain: /content/yolov5/Gatto-1/training/images\nval: /content/yolov5/Gatto-1/validation/images\n")
data.close()

# Run the YOLOv5 (we used a yolov5s)

In [ ]:
!python train.py --img 416 --batch 50 --epochs 300 --data /content/yolov5/Gatto-1/data.yaml --weights yolov5s.pt --cache

### Fit the model (by using the best.pt weights you have just got)

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', 
                       path='/content/yolov5/runs/train/exp/weights/best.pt', force_reload=True)

model.conf = 0.25
model.iou = 0.45
model.agnostic = False
model.multi_label = False
model.classes = None
model.max_det = 1000
model.amp = False

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # This is pretrained (200000 pictures)

### Test the model on the testing set

In [ ]:
for filename in os.listdir('/content/yolov5/Gatto-1/test/images'):
  img = Image.open('/content/yolov5/Gatto-1/test/images/' + filename)
  results = model(img)
  results.save()

In [ ]:
print(results.xywhn) # x center, y center, width, height, probability, class (ALL SCALED TO 1)
print(results.xywh) # x center, y center, width, height, probability, class (ALL NOT SCALED TO 1)

[tensor([[0.60287, 0.26370, 0.31938, 0.32483, 0.87996, 0.00000]])]
[tensor([[250.79568, 109.69796, 132.86078, 135.13115,   0.87996,   0.00000]])]
